In [32]:
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from google.cloud import bigquery
import requests
import pandas as pd
response = requests.get(url = "https://api.glassnode.com/v1/metrics/addresses/active_count",
                       params = {'a':'BTC', 'api_key':'25vXXzFYhvEVJBXOGSApnZaDR4D'})
response = response.json()

In [33]:
active_holding = pd.DataFrame(response)
active_holding.head(3)

* t is the timestamp
* v is the value column

### Converting TimeStamp Column to Date Time

In [34]:
active_holding['t'] = pd.to_datetime(active_holding['t'], unit = 's')
active_holding.head(3)

### Fetch Data for the Market Cap of BitCoin

In [35]:
response = requests.get(url = "https://api.glassnode.com/v1/metrics/market/marketcap_usd",
                       params = {'a':'BTC', 'api_key':'25vXXzFYhvEVJBXOGSApnZaDR4D'})
response = response.json()

market_cap = pd.DataFrame(response)
market_cap['t'] =  pd.to_datetime(market_cap['t'], unit = 's')
market_cap.head(3)

### Comparing Shape between Active Holding and Market Cap

In [36]:
active_holding.shape, market_cap.shape

### Use Inner Join to Merge the Two Tables

In [37]:
additional_data = active_holding.merge(market_cap, how = 'inner', on = 't')

In [38]:
additional_data.rename(columns = {'t':'time', 'v_x': 'active_address', 'v_y': 'market_cap'}, inplace = True)
additional_data.head()

In [39]:
def load_dataset_from_query(client, query_):
    job = client.query(query_)
    return job.to_dataframe()

In [40]:
def load_data_from_ref_object(client, dataset_name, table_name, max_results = 10):
    dataset_ref = client.dataset(dataset_name)
    dataset = client.get_dataset(dataset_ref)
    
    print("List of all Tables in Dataset --")
    tables = list(client.list_tables(dataset))
    for t in tables:
        print(t.table_id)
        
    table_ref =  dataset_ref.table(table_name)
    table = client.get_table(table_ref)
    
    print("Schema of the Table --")
    print(table.schema)
    data = client.list_rows(table, max_results = max_results).to_dataframe()
    return data

### Fetching Main Data from Public DataSet

In [41]:
#Give your Dataset and Table Name
client = bigquery.Client(project = "aqueous-nebula-343009")
main_data = load_data_from_ref_object(client,"bitcoin_blockhain", "blocks")
main_data.head()

In [42]:
#Setup client project id
QUERY =  "SELECT DATE(TIMESTAMP_MILLIS(timestamp)) AS time, AVG(difficultyTarget) as AvgDifficultyThreshold, AVG(work_terahash) as TeraHash FROM aqueous-nebula-343009.bitcoin_blockhain.blocks GROUP BY time ORDER BY time"
main_data = load_dataset_from_query(client, QUERY)
main_data.head()

In [43]:
main_data['time'] = pd.to_datetime(main_data['time'])

### Joining All Data

In [44]:
main_data = main_data.merge(additional_data, how = 'inner', on = 'time')
main_data.head()

### Plotting the Graphs

In [45]:
fig, axs = plt.subplots(2,2, figsize = (13,8))

axs[0,0].plot(main_data['TeraHash'])
axs[0,0].set_title('HashRate')

axs[0,1].plot(main_data['active_address'])
axs[0,1].set_title('ActiveAddress')

axs[1,0].plot(main_data['market_cap'])
axs[1,0].set_title('MarketCap')

axs[1,1].plot(main_data['AvgDifficultyThreshold'])
axs[1,1].set_title('Difficulty')

### What's the most difficult block mined and when?

In [47]:
#Query to get the most difficult block mined
QUERY = "SELECT block_id, difficultyTarget FROM aqueous-nebula-343009.bitcoin_blockhain.blocks WHERE difficultyTarget = (SELECT MIN(difficultyTarget) FROM aqueous-nebula-343009.bitcoin_blockhain.blocks) LIMIT 1"
difficult_mine = load_dataset_from_query(client, QUERY)

In [55]:
difficult_block = difficult_mine.block_id.values[0]
difficult_block

In [70]:
#Date the Block was mined
QUERY = "SELECT block_id, difficultyTarget, DATE(TIMESTAMP_MILLIS(timestamp)) AS time FROM aqueous-nebula-343009.bitcoin_blockhain.blocks WHERE  block_id LIKE  '%23fbfb8aa9a6e4a47c852656c91d11d2500ed5f5ec981e'" 
difficult_mine_time = load_dataset_from_query(client, QUERY)

In [74]:
time_dm = difficult_mine_time['time'].values[0].strftime('%Y-%m-%d')
time_dm

In [75]:
main_data[main_data['time'] == time_dm ]